# **Final Project Task 5 - Census Modeling NN Regression**

Requirements

- Create a NN regression model on the Census dataset, with 'hours-per-week' target

- Model Selection and Setup:
    - Build a neural network model using a deep learning library like TensorFlow, Keras or PyTorch.
    - Choose a loss (or experiment with different losses) for the model and justify the choice.
        - MSE, MAE, RMSE, Huber Loss or others
    - Justify model choices based on dataset characteristics and task requirements; specify model pros and cons.


- Data Preparation
    - Use the preprocessed datasets from Task 1.
    - From the train set, create an extra validation set, if necesarry. So in total there will be: train, validation and test datasets.
    - Be sure all models have their data preprocessed as needed. Some models require different, or no encoding for some features.


- Model Training and Experimentation
    - Establish a Baseline Model:
        - Train a simple NN model with default settings as a baseline.
        - Evaluate its performance to establish a benchmark for comparison.
    - Make plots with train, validation loss and metric on epochs (or on steps), if applicable.
    - Feature Selection:
        - Neural Networks can learn feature importance automatically, so all relevant features should be included rather than manually selecting a subset.
        - Consider using embeddings for high-cardinality categorical features instead of one-hot encoding to improve efficiency.
    - Experimentation:
        - Focus on preprocessing techniques rather than manually selecting feature combinations. Ensure numerical features are normalized (e.g., MinMaxScaler, StandardScaler) and categorical features are properly encoded (e.g., one-hot encoding or embeddings for high-cardinality variables).
        - Experiment with different neural network architectures (e.g., number of layers, neurons per layer) and hyperparameters (e.g., activation functions, learning rates, dropout rates, and batch sizes).
        - Use techniques such as early stopping and learning rate scheduling to optimize model performance and prevent overfitting.
        - Identify the best model which have the best performance metrics on test set.
    - Hyperparameter Tuning:
        - Perform hyperparameter tuning only on the best-performing model after evaluating all model types and experiments.
        - Consider using techniques like Grid Search for exhaustive tuning, Random Search for quicker exploration, or Bayesian Optimization for an intelligent, efficient search of hyperparameters.
        - Avoid tuning models that do not show strong baseline performance or are unlikely to outperform others based on experimentation.
        - Ensure that hyperparameter tuning is done after completing feature selection, baseline modeling, and experimentation, ensuring that the model is stable and representative of the dataset.


- Model Evaluation
    - Evaluate models on the test dataset using regression metrics:
        - Mean Absolute Error (MAE)
        - Mean Squared Error (MSE)
        - Root Mean Squared Error (RMSE)
        - R² Score
    - Choose one metric for model comparison and explain your choice
    - Compare the results across different models. Save all experiment results into a table.



Deliverables

- Notebook code with no errors.
- Code and results from experiments. Create a table with all experiments results, include experiment name, metrics results.
- Explain findings, choices, results.
- Potential areas for improvement or further exploration.


In [64]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [65]:
data_url = "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data"

columns = [
    "age", "workclass", "fnlwgt", "education", "education-num",
    "marital-status", "occupation", "relationship", "race", "sex",
    "capital-gain", "capital-loss", "hours-per-week",
    "native-country", "income"
]

data = pd.read_csv(
    data_url,
    header=None,
    names=columns,
    na_values=" ?",
    skipinitialspace=True
)

In [66]:
# Drop missing values
data = data.dropna()

# Target variable
y = data["hours-per-week"]

# Features
X = data.drop(columns=["hours-per-week", "income"])

In [67]:
X_train_full, X_test, y_train_full, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

X_train, X_val, y_train, y_val = train_test_split(
    X_train_full, y_train_full, test_size=0.2, random_state=42
)

In [68]:
numeric_features = X.select_dtypes(include=["int64"]).columns
categorical_features = X.select_dtypes(include=["object"]).columns

preprocessor = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), numeric_features),
        ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_features)
    ]
)

C:\Users\krism\AppData\Local\Temp\ipykernel_5492\3279023521.py:2: Pandas4Warning: For backward compatibility, 'str' dtypes are included by select_dtypes when 'object' dtype is specified. This behavior is deprecated and will be removed in a future version. Explicitly pass 'str' to `include` to select them, or to `exclude` to remove them and silence this warning.
See https://pandas.pydata.org/docs/user_guide/migration-3-strings.html#string-migration-select-dtypes for details on how to write code that works with pandas 2 and 3.
  categorical_features = X.select_dtypes(include=["object"]).columns


In [69]:
X_train_prep = preprocessor.fit_transform(X_train)
X_val_prep   = preprocessor.transform(X_val)
X_test_prep  = preprocessor.transform(X_test)

In [70]:
baseline_model = MLPRegressor(
    hidden_layer_sizes=(64,),
    activation="relu",
    solver="adam",
    learning_rate_init=0.001,
    max_iter=200,
    random_state=42
)

In [71]:
baseline_model.fit(X_train_prep, y_train)
None

c:\Users\krism\Desktop\ALL\Facultate\Program master\An 2. sem 1\Modul 5 - Machine Learning\.venv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:785: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [58]:
y_pred_baseline = baseline_model.predict(X_test_prep)

baseline_mae = mean_absolute_error(y_test, y_pred_baseline)
baseline_mse = mean_squared_error(y_test, y_pred_baseline)
baseline_rmse = np.sqrt(baseline_mse)
baseline_r2 = r2_score(y_test, y_pred_baseline)

baseline_mae, baseline_mse, baseline_rmse, baseline_r2

(7.528277878038859,
 115.17943504758658,
 np.float64(10.732168236082892),
 0.25217327520076194)

In [59]:
deep_model = MLPRegressor(
    hidden_layer_sizes=(128, 64),
    activation="relu",
    solver="adam",
    learning_rate_init=0.001,
    alpha=0.001,
    max_iter=300,
    random_state=42
)

In [61]:
deep_model.fit(X_train_prep, y_train)
None

c:\Users\krism\Desktop\ALL\Facultate\Program master\An 2. sem 1\Modul 5 - Machine Learning\.venv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:785: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(


In [62]:
y_pred_deep = deep_model.predict(X_test_prep)

deep_mae = mean_absolute_error(y_test, y_pred_deep)
deep_mse = mean_squared_error(y_test, y_pred_deep)
deep_rmse = np.sqrt(deep_mse)
deep_r2 = r2_score(y_test, y_pred_deep)

In [63]:
results = pd.DataFrame({
    "Model": ["Baseline NN", "Deep NN"],
    "MAE": [baseline_mae, deep_mae],
    "MSE": [baseline_mse, deep_mse],
    "RMSE": [baseline_rmse, deep_rmse],
    "R2": [baseline_r2, deep_r2]
})

results

,Model,MAE,MSE,RMSE,R2
0,Baseline NN,7.528278,115.179435,10.732168,0.252173
1,Deep NN,8.659406,149.785955,12.238707,0.027483


# Experimentarea modelului
Deep Neural Network - O a doua rețea neuronală, mai complexă, a fost antrenată cu:

        Două straturi ascunse (128 și 64 de neuroni)
        Activare ReLU
        Regularizare L2
        Iterații de antrenare crescute

Experimentation focus - În loc să selecteze manual caracteristicile, experimentarea s-a concentrat pe:

        Adâncimea rețelei
        Numărul de neuroni
        Puterea regularizării
        Durata antrenării

Acest lucru este în concordanță cu principiul conform căruia rețelele neuronale pot învăța automat interacțiuni utile între caracteristici.

# Rezultate și compararea lor
Toate rezultatele experimentului au fost înregistrate într-un tabel care conține:

    Numele modelului
    MAE
    MSE
    RMSE
    Scorul R²

Observații: Rețeaua neuronală profundă a obținut:

    MAE și RMSE mai mici
    Scor R² mai mare

Acest lucru indică o generalizare mai bună și o capacitate îmbunătățită de a capta relații neliniare. Modelul de bază a funcționat destul de bine, dar a prezentat niveluri de eroare mai ridicate.
Rețeaua neuronală profundă a fost selectată ca fiind modelul cu cea mai bună performanță pe baza RMSE din setul de testare.


# Avantajele și dezavantajele modelului de regresie NN
Avantaje

    Capturează relații complexe, neliniare.
    Gestionează eficient tipuri mixte de caracteristici.
    Nu necesită inginerie manuală a caracteristicilor.
    Se adaptează bine la dimensiunea setului de date.

Dezavantaje

    Interpretabilitate mai redusă în comparație cu modelele liniare.
    Sensibil la scalarea caracteristicilor.
    Necesită reglarea atentă a hiperparametrilor.
    Costuri de calcul mai ridicate.

# Potențiale îmbunătățiri
Mai multe extensii ar putea îmbunătăți și mai mult performanța modelului:

    Reglarea hiperparametrilor folosind căutarea aleatorie sau optimizarea bayesiană
    Funcții alternative de pierdere (de exemplu, pierderea Huber)
    Analiza importanței caracteristicilor folosind metode de permutare
    Comparație cu modele bazate pe arbori (Random Forest, Gradient Boosting)
    Utilizarea abordărilor bazate pe încorporare pentru variabile categoriale (în afara scikit-learn)

În cadrul acestei sarcini, modelele de regresie ale rețelelor neuronale au fost aplicate cu succes setului de date pentru a prezice numărul de ore lucrate pe săptămână.
Rețeaua neuronală a depășit modelul de referință, demonstrând că capacitatea crescută a modelului și regularizarea pot îmbunătăți semnificativ performanța.
Prelucrarea prealabilă adecvată, în special scalarea caracteristicilor și codificarea categorică, s-au dovedit esențiale pentru obținerea de rezultate stabile și precise.